## Data Cleaning

In [277]:
import requests, bs4
from bs4 import BeautifulSoup as bs
import pandas as pd
from datetime import datetime 
import statsmodels.api as sm
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import csv

### This section is used to clean all the data to prepare for regression analysis

In [278]:
df = pd.read_csv('project_data2/regression_data.csv') 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2400 entries, 0 to 2399
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Title                2400 non-null   object 
 1   Domestic_gross       2400 non-null   int64  
 2   International_gross  2400 non-null   int64  
 3   Worldwide            2395 non-null   float64
 4   theater              2400 non-null   object 
 5   release_date         2400 non-null   object 
 6   Release_Period       2131 non-null   object 
 7   distributer          2400 non-null   object 
 8   Genres               2396 non-null   object 
 9   MPAA                 2219 non-null   object 
 10  Running_Time         2397 non-null   object 
 11  Budget               1389 non-null   float64
dtypes: float64(2), int64(2), object(8)
memory usage: 225.1+ KB


In [279]:
df.head()

,Title,Domestic_gross,International_gross,Worldwide,theater,release_date,Release_Period,distributer,Genres,MPAA,Running_Time,Budget
0,Avatar,749766139,1993811448,2.743578e+09,3461,Dec 18 2010,Dec 18 2009 - Aug 12 2010,Twentieth Century Fox,Action Adventure Fantasy...,PG-13,2 hr 42 min,237000000.0
1,Toy Story 3,415004880,651964823,1.066970e+09,4028,Jun 18 2010,Jun 18 2010 - Dec 2 2010,Walt Disney Studios Motion Pictures,Adventure Animation Come...,NaN,1 hr 43 min,200000000.0
2,Alice in Wonderland,334191110,691276000,1.025467e+09,3739,Mar 5 2010,Mar 5 2010 - Jul 8 2010,Walt Disney Studios Motion Pictures,Adventure Family Fantasy...,PG,1 hr 48 min,200000000.0
3,Iron Man 2,312433331,311500000,6.239333e+08,4390,May 7 2010,May 7 2010 - Aug 19 2010,Paramount Pictures,Action Adventure Sci-Fi,PG-13,2 hr 4 min,200000000.0
4,The Twilight Saga: Eclipse,300531751,397959596,6.984913e+08,4468,Jun 30 2010,Jun 30 2010 - Oct 21 2010,Summit Entertainment,Action Adventure Drama ...,PG-13,2 hr 4 min,68000000.0


### Get the Release Period length

In [280]:
df['Release_End'] = df['Release_Period']
df['Release_Start'] = df['Release_Period']

In [281]:
for i in range(len(df['Release_Period'])):
    try:
        df['Release_End'][i] = df['Release_Period'][i].split("-",1)[1].strip()
    except:
        df['Release_End'][i] = np.nan
        
for i in range(len(df['Release_Period'])):        
    try:
        df['Release_Start'][i] = df['Release_Period'][i].split("-",1)[0].strip()
    except:
        df['Release_Start'][i] = np.nan

/tmp/ipykernel_30728/1172296270.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Release_End'][i] = df['Release_Period'][i].split("-",1)[1].strip()
/tmp/ipykernel_30728/1172296270.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Release_End'][i] = np.nan
/tmp/ipykernel_30728/1172296270.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Release_Start'][i] = df['Release_Period'][i].split("-",1)[0].strip()
/tmp/ipykerne

In [282]:
df.head()

,Title,Domestic_gross,International_gross,Worldwide,theater,release_date,Release_Period,distributer,Genres,MPAA,Running_Time,Budget,Release_End,Release_Start
0,Avatar,749766139,1993811448,2.743578e+09,3461,Dec 18 2010,Dec 18 2009 - Aug 12 2010,Twentieth Century Fox,Action Adventure Fantasy...,PG-13,2 hr 42 min,237000000.0,Aug 12 2010,Dec 18 2009
1,Toy Story 3,415004880,651964823,1.066970e+09,4028,Jun 18 2010,Jun 18 2010 - Dec 2 2010,Walt Disney Studios Motion Pictures,Adventure Animation Come...,NaN,1 hr 43 min,200000000.0,Dec 2 2010,Jun 18 2010
2,Alice in Wonderland,334191110,691276000,1.025467e+09,3739,Mar 5 2010,Mar 5 2010 - Jul 8 2010,Walt Disney Studios Motion Pictures,Adventure Family Fantasy...,PG,1 hr 48 min,200000000.0,Jul 8 2010,Mar 5 2010
3,Iron Man 2,312433331,311500000,6.239333e+08,4390,May 7 2010,May 7 2010 - Aug 19 2010,Paramount Pictures,Action Adventure Sci-Fi,PG-13,2 hr 4 min,200000000.0,Aug 19 2010,May 7 2010
4,The Twilight Saga: Eclipse,300531751,397959596,6.984913e+08,4468,Jun 30 2010,Jun 30 2010 - Oct 21 2010,Summit Entertainment,Action Adventure Drama ...,PG-13,2 hr 4 min,68000000.0,Oct 21 2010,Jun 30 2010


In [283]:
df['release_endtime'] = df['Release_Period']
df['release_starttime'] = df['Release_Period']

In [284]:
for i in range(len(df['Release_End'])):
    try:
        df['release_starttime'][i] = datetime.strptime(df['Release_Start'][i], '%b %d %Y')
    except:
        df['release_starttime'][i] = np.nan


/tmp/ipykernel_30728/445816969.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['release_starttime'][i] = datetime.strptime(df['Release_Start'][i], '%b %d %Y')
/tmp/ipykernel_30728/445816969.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['release_starttime'][i] = np.nan


In [285]:
for i in range(len(df['Release_End'])):
    try:
        df['release_endtime'][i] = datetime.strptime(df['Release_End'][i], '%b %d %Y')
    except:
        df['release_endtime'][i] = np.nan

/tmp/ipykernel_30728/2496302557.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['release_endtime'][i] = datetime.strptime(df['Release_End'][i], '%b %d %Y')
/tmp/ipykernel_30728/2496302557.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['release_endtime'][i] = np.nan


In [286]:
df.head()

,Title,Domestic_gross,International_gross,Worldwide,theater,release_date,Release_Period,distributer,Genres,MPAA,Running_Time,Budget,Release_End,Release_Start,release_endtime,release_starttime
0,Avatar,749766139,1993811448,2.743578e+09,3461,Dec 18 2010,Dec 18 2009 - Aug 12 2010,Twentieth Century Fox,Action Adventure Fantasy...,PG-13,2 hr 42 min,237000000.0,Aug 12 2010,Dec 18 2009,2010-08-12 00:00:00,2009-12-18 00:00:00
1,Toy Story 3,415004880,651964823,1.066970e+09,4028,Jun 18 2010,Jun 18 2010 - Dec 2 2010,Walt Disney Studios Motion Pictures,Adventure Animation Come...,NaN,1 hr 43 min,200000000.0,Dec 2 2010,Jun 18 2010,2010-12-02 00:00:00,2010-06-18 00:00:00
2,Alice in Wonderland,334191110,691276000,1.025467e+09,3739,Mar 5 2010,Mar 5 2010 - Jul 8 2010,Walt Disney Studios Motion Pictures,Adventure Family Fantasy...,PG,1 hr 48 min,200000000.0,Jul 8 2010,Mar 5 2010,2010-07-08 00:00:00,2010-03-05 00:00:00
3,Iron Man 2,312433331,311500000,6.239333e+08,4390,May 7 2010,May 7 2010 - Aug 19 2010,Paramount Pictures,Action Adventure Sci-Fi,PG-13,2 hr 4 min,200000000.0,Aug 19 2010,May 7 2010,2010-08-19 00:00:00,2010-05-07 00:00:00
4,The Twilight Saga: Eclipse,300531751,397959596,6.984913e+08,4468,Jun 30 2010,Jun 30 2010 - Oct 21 2010,Summit Entertainment,Action Adventure Drama ...,PG-13,2 hr 4 min,68000000.0,Oct 21 2010,Jun 30 2010,2010-10-21 00:00:00,2010-06-30 00:00:00


In [287]:
df['Release_Period_Time_days'] = df['Release_Period']
for i in range(len(df['Release_End'])):
    try:
        df['Release_Period_Time_days'][i] = abs(df['release_starttime'][i]-df['release_endtime'][i]).days
    except:
        df['Release_Period_Time_days'][i] = np.nan

/tmp/ipykernel_30728/4137098046.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Release_Period_Time_days'][i] = abs(df['release_starttime'][i]-df['release_endtime'][i]).days
/tmp/ipykernel_30728/4137098046.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Release_Period_Time_days'][i] = np.nan


In [288]:
df.head()

,Title,Domestic_gross,International_gross,Worldwide,theater,release_date,Release_Period,distributer,Genres,MPAA,Running_Time,Budget,Release_End,Release_Start,release_endtime,release_starttime,Release_Period_Time_days
0,Avatar,749766139,1993811448,2.743578e+09,3461,Dec 18 2010,Dec 18 2009 - Aug 12 2010,Twentieth Century Fox,Action Adventure Fantasy...,PG-13,2 hr 42 min,237000000.0,Aug 12 2010,Dec 18 2009,2010-08-12 00:00:00,2009-12-18 00:00:00,237
1,Toy Story 3,415004880,651964823,1.066970e+09,4028,Jun 18 2010,Jun 18 2010 - Dec 2 2010,Walt Disney Studios Motion Pictures,Adventure Animation Come...,NaN,1 hr 43 min,200000000.0,Dec 2 2010,Jun 18 2010,2010-12-02 00:00:00,2010-06-18 00:00:00,167
2,Alice in Wonderland,334191110,691276000,1.025467e+09,3739,Mar 5 2010,Mar 5 2010 - Jul 8 2010,Walt Disney Studios Motion Pictures,Adventure Family Fantasy...,PG,1 hr 48 min,200000000.0,Jul 8 2010,Mar 5 2010,2010-07-08 00:00:00,2010-03-05 00:00:00,125
3,Iron Man 2,312433331,311500000,6.239333e+08,4390,May 7 2010,May 7 2010 - Aug 19 2010,Paramount Pictures,Action Adventure Sci-Fi,PG-13,2 hr 4 min,200000000.0,Aug 19 2010,May 7 2010,2010-08-19 00:00:00,2010-05-07 00:00:00,104
4,The Twilight Saga: Eclipse,300531751,397959596,6.984913e+08,4468,Jun 30 2010,Jun 30 2010 - Oct 21 2010,Summit Entertainment,Action Adventure Drama ...,PG-13,2 hr 4 min,68000000.0,Oct 21 2010,Jun 30 2010,2010-10-21 00:00:00,2010-06-30 00:00:00,113


In [289]:
df['Running_Time_Length_seconds'] = df['Budget']
for i in range(len(df['Running_Time'])):
    mid = str(df['Running_Time'][i]).replace('min','')
    try:
        mid2 = float(mid.split("hr",1)[1].strip())
    except:
        mid2 = 0.0
    df['Running_Time_Length_seconds'][i] = float(mid.split("hr",1)[0].strip())*60+mid2

/tmp/ipykernel_30728/52295990.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Running_Time_Length_seconds'][i] = float(mid.split("hr",1)[0].strip())*60+mid2


In [290]:
df['Genres_thefirst'] = df['Genres']
for i in range(len(df['Genres'])):
    
    df['Genres_thefirst'][i] = str(df['Genres'][i]).split(" ",1)[0]

/tmp/ipykernel_30728/1553883654.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Genres_thefirst'][i] = str(df['Genres'][i]).split(" ",1)[0]


In [291]:
df.head()

,Title,Domestic_gross,International_gross,Worldwide,theater,release_date,Release_Period,distributer,Genres,MPAA,Running_Time,Budget,Release_End,Release_Start,release_endtime,release_starttime,Release_Period_Time_days,Running_Time_Length_seconds,Genres_thefirst
0,Avatar,749766139,1993811448,2.743578e+09,3461,Dec 18 2010,Dec 18 2009 - Aug 12 2010,Twentieth Century Fox,Action Adventure Fantasy...,PG-13,2 hr 42 min,237000000.0,Aug 12 2010,Dec 18 2009,2010-08-12 00:00:00,2009-12-18 00:00:00,237,162.0,Action
1,Toy Story 3,415004880,651964823,1.066970e+09,4028,Jun 18 2010,Jun 18 2010 - Dec 2 2010,Walt Disney Studios Motion Pictures,Adventure Animation Come...,NaN,1 hr 43 min,200000000.0,Dec 2 2010,Jun 18 2010,2010-12-02 00:00:00,2010-06-18 00:00:00,167,103.0,Adventure
2,Alice in Wonderland,334191110,691276000,1.025467e+09,3739,Mar 5 2010,Mar 5 2010 - Jul 8 2010,Walt Disney Studios Motion Pictures,Adventure Family Fantasy...,PG,1 hr 48 min,200000000.0,Jul 8 2010,Mar 5 2010,2010-07-08 00:00:00,2010-03-05 00:00:00,125,108.0,Adventure
3,Iron Man 2,312433331,311500000,6.239333e+08,4390,May 7 2010,May 7 2010 - Aug 19 2010,Paramount Pictures,Action Adventure Sci-Fi,PG-13,2 hr 4 min,200000000.0,Aug 19 2010,May 7 2010,2010-08-19 00:00:00,2010-05-07 00:00:00,104,124.0,Action
4,The Twilight Saga: Eclipse,300531751,397959596,6.984913e+08,4468,Jun 30 2010,Jun 30 2010 - Oct 21 2010,Summit Entertainment,Action Adventure Drama ...,PG-13,2 hr 4 min,68000000.0,Oct 21 2010,Jun 30 2010,2010-10-21 00:00:00,2010-06-30 00:00:00,113,124.0,Action


In [295]:
df_new = df.iloc[: , [0,1,2,3,4,7,18,9,11,15,16,17]].copy() 

In [296]:
df_new.head()

,Title,Domestic_gross,International_gross,Worldwide,theater,distributer,Genres_thefirst,MPAA,Budget,release_starttime,Release_Period_Time_days,Running_Time_Length_seconds
0,Avatar,749766139,1993811448,2.743578e+09,3461,Twentieth Century Fox,Action,PG-13,237000000.0,2009-12-18 00:00:00,237,162.0
1,Toy Story 3,415004880,651964823,1.066970e+09,4028,Walt Disney Studios Motion Pictures,Adventure,NaN,200000000.0,2010-06-18 00:00:00,167,103.0
2,Alice in Wonderland,334191110,691276000,1.025467e+09,3739,Walt Disney Studios Motion Pictures,Adventure,PG,200000000.0,2010-03-05 00:00:00,125,108.0
3,Iron Man 2,312433331,311500000,6.239333e+08,4390,Paramount Pictures,Action,PG-13,200000000.0,2010-05-07 00:00:00,104,124.0
4,The Twilight Saga: Eclipse,300531751,397959596,6.984913e+08,4468,Summit Entertainment,Action,PG-13,68000000.0,2010-06-30 00:00:00,113,124.0


In [306]:
df_new.dropna(inplace = True)

In [307]:
df_data = df_new.reset_index(drop=True)

In [313]:
df_data['release_year'] = pd.DatetimeIndex(df_data['release_starttime']).year
df_data['release_month'] = pd.DatetimeIndex(df_data['release_starttime']).month
df_data['release_weekday'] = df_data['release_starttime']
for i in range(len(df_data['release_starttime'])):
    df_data['release_weekday'][i] = df_data['release_starttime'][i].weekday()
df_data.drop('release_starttime',1,inplace=True)

/tmp/ipykernel_30728/2871209387.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data['release_weekday'][i] = df_data['release_starttime'][i].weekday()
/tmp/ipykernel_30728/2871209387.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df_data.drop('release_starttime',1,inplace=True)


In [314]:
df_data.head()

,Title,Domestic_gross,International_gross,Worldwide,theater,distributer,Genres_thefirst,MPAA,Budget,Release_Period_Time_days,Running_Time_Length_seconds,release_year,release_month,release_weekday
0,Avatar,749766139,1993811448,2.743578e+09,3461,Twentieth Century Fox,Action,PG-13,237000000.0,237,162.0,2009,12,4
1,Alice in Wonderland,334191110,691276000,1.025467e+09,3739,Walt Disney Studios Motion Pictures,Adventure,PG,200000000.0,125,108.0,2010,3,4
2,Iron Man 2,312433331,311500000,6.239333e+08,4390,Paramount Pictures,Action,PG-13,200000000.0,104,124.0,2010,5,4
3,The Twilight Saga: Eclipse,300531751,397959596,6.984913e+08,4468,Summit Entertainment,Action,PG-13,68000000.0,113,124.0,2010,6,2
4,Inception,292576195,533560993,8.261372e+08,3792,Warner Bros.,Action,PG-13,160000000.0,174,148.0,2010,7,4


In [315]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1174 entries, 0 to 1173
Data columns (total 14 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Title                        1174 non-null   object 
 1   Domestic_gross               1174 non-null   int64  
 2   International_gross          1174 non-null   int64  
 3   Worldwide                    1174 non-null   float64
 4   theater                      1174 non-null   object 
 5   distributer                  1174 non-null   object 
 6   Genres_thefirst              1174 non-null   object 
 7   MPAA                         1174 non-null   object 
 8   Budget                       1174 non-null   float64
 9   Release_Period_Time_days     1174 non-null   object 
 10  Running_Time_Length_seconds  1174 non-null   float64
 11  release_year                 1174 non-null   int64  
 12  release_month                1174 non-null   int64  
 13  release_weekday   

In [316]:
df_data.to_csv('project_data2/regression_data_cleand.csv',index = False)